In [9]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA



In [10]:
## Read the ppdfs from the folder
loader=PyPDFDirectoryLoader("./us_census")

documents=loader.load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

final_documents=text_splitter.split_documents(documents)
final_documents

[Document(metadata={'source': 'us_census\\acsbr-015.pdf', 'page': 0}, page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015\nIssued September 2023\nDouglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to \nhealth coverage. For example, between 2021 and 2022, \nthe labor market continued to improve, which may \nhave affected private coverage in the United States \nduring that time.1 Public policy changes included \nthe renewal of the Public Health Emergency, which \nallowed Medicaid enrollees to remain covered under \nthe Continuous Enrollment Provision.2 The American \nRescue Plan (ARP) enhanced Marketplace premium \nsubsidies for those with incomes above 400 percent \nof the poverty level as well as for unemployed people.3\nIn addition to national policies, individual states and \nthe District of Columbia can affect

In [11]:
len(final_documents)

316

In [12]:
## Embedding Using Huggingface
huggingface_embeddings=HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",      #sentence-transformers/all-MiniLM-l6-v2
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}

)

In [13]:
import  numpy as np
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)))
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)).shape)

[-0.07903481 -0.01134114 -0.02312096  0.02844463  0.05053342  0.05317826
 -0.01907787  0.03456026 -0.10211365 -0.02915703  0.08524261  0.05650725
 -0.02545439 -0.03308488 -0.00635736  0.0409086  -0.00628108  0.00356743
 -0.03854128  0.03667684 -0.04289802  0.03425247 -0.03116898 -0.03793728
  0.01728389  0.01214923  0.0065312   0.01463561 -0.05529054 -0.15320708
  0.00730849  0.03202945 -0.04701126 -0.01595976  0.01874445  0.02642932
 -0.02306377  0.08438039  0.04182488  0.05278176 -0.03057603  0.01564259
 -0.01689072  0.0052941  -0.02417435  0.00412996 -0.01889938 -0.00150624
 -0.00836943 -0.03390064  0.03515956 -0.00553129  0.04910937  0.05971859
  0.05615961 -0.05105155  0.01475137 -0.01849957 -0.03284641  0.03576626
  0.04947705 -0.00938883 -0.26202115  0.09750333  0.01715693  0.04781387
 -0.00556317 -0.00298305 -0.02207354 -0.04463668 -0.05760484  0.04815875
 -0.0552221   0.01635334  0.03299246  0.02147076  0.01296215  0.0146231
  0.02174954 -0.00202998  0.02099538  0.03353843 -0.

In [14]:
## VectorStore Creation
vectorstore=FAISS.from_documents(final_documents[:120],huggingface_embeddings)

In [15]:
## Query using Similarity Search
query="WHAT IS HEALTH INSURANCE COVERAGE?"
relevant_docments=vectorstore.similarity_search(query)

print(relevant_docments[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [16]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000012E3C52DA90> search_kwargs={'k': 3}


In [17]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['HUGGINGFACEHUB_API_TOKEN']= os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [12]:
from langchain_community.llms import HuggingFaceHub

hf=HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-v0.1",
    model_kwargs={"temperature":0.1,"max_length":500}

)
query="What is the health insurance coverage?"
hf.invoke(query)

C:\Users\srija\AppData\Local\Temp\ipykernel_9724\386166796.py:3: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpoint`.
  hf=HuggingFaceHub(


KeyboardInterrupt: 

In [6]:
#Hugging Face models can be run locally through the HuggingFacePipeline class.
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="EleutherAI/gpt-neo-125M",
    task="text-generation",
    pipeline_kwargs={"temperature": 0, "max_new_tokens": 300}
)
query = "Explain the basics of quantum mechanics."

llm = hf 
llm.invoke(query)

f:\LangChain\venv\lib\site-packages\transformers\generation\configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


'Explain the basics of quantum mechanics.\n\nThe quantum mechanics of the universe is a complex system of laws that are based on the laws of physics. The laws of physics are the laws of physics. The laws of physics are the laws of physics. The laws of physics are the laws of physics. The laws of physics are the laws of physics. The laws of physics are the laws of physics. The laws of physics are the laws of physics. The laws of physics are the laws of physics. The laws of physics are the laws of physics. The laws of physics are the laws of physics. The laws of physics are the laws of physics. The laws of physics are the laws of physics. The laws of physics are the laws of physics. The laws of physics are the laws of physics. The laws of physics are the laws of physics. The laws of physics are the laws of physics. The laws of physics are the laws of physics. The laws of physics are the laws of physics. The laws of physics are the laws of physics. The laws of physics are the laws of phys

In [7]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [18]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [19]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [22]:
query="""what is health care insaurance coverage"""

In [23]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])

f:\LangChain\venv\lib\site-packages\transformers\generation\configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.
gov), or coverage through TRICARE. Public insurance coverage 
includes federal programs (such as Medicare, Medicaid, and the 
Children’s Health Insurance Program or CHIP), individual state 
health plans, and CHAMPVA (Civilian Health and Medical Program 
at the Department of Veterans Affairs), as well as care provided 
by the Department of Veterans Affairs. In the ACS, people are 
considered insured if they were covered by any of these types 
of health insurance at time of interview. People are considered 
uninsured if they were not covered by any of these types of health 
insurance at time of interview or if they only had coverage through 
the Indian Health Service (IHS), 